In [108]:
import math
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory

import numpy as np
import pandas as pd
from rdkit.Chem import PandasTools
from chembl_webresource_client.new_client import new_client
from tqdm.auto import tqdm

In [109]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

In [110]:
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_classification', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


In [111]:
targets_api = new_client.target
compounds_api = new_client.molecule
bioactivities_api = new_client.activity
assay_api = new_client.assay
protein_api = new_client.protein_classification
binding_api = new_client.binding_site
biotherapeutic_api = new_client.biotherapeutic
mechanism_api = new_client.mechanism

In [116]:
data = pd.read_csv("144_compounds_updated.csv")
dataset = pd.DataFrame(data)
cmps= dataset["Column1"].dropna().values.tolist()
compounds = dict.fromkeys(dataset["Column1"].dropna().values.tolist(),{"target":[], "bioactivities":[], "assay":[]})
compounds
print(compounds)



{'Tacalcitol': {'target': [], 'bioactivities': [], 'assay': []}, 'Penfluridol': {'target': [], 'bioactivities': [], 'assay': []}, 'AVL-292': {'target': [], 'bioactivities': [], 'assay': []}, 'HMN-214': {'target': [], 'bioactivities': [], 'assay': []}, 'Canertinib': {'target': [], 'bioactivities': [], 'assay': []}, 'RG7112': {'target': [], 'bioactivities': [], 'assay': []}, 'Navitoclax': {'target': [], 'bioactivities': [], 'assay': []}, 'UNII-BXU45ZH6LI': {'target': [], 'bioactivities': [], 'assay': []}, 'LY2090314': {'target': [], 'bioactivities': [], 'assay': []}, 'CEP-37440': {'target': [], 'bioactivities': [], 'assay': []}, 'Alogliptin': {'target': [], 'bioactivities': [], 'assay': []}, 'CHIR-99021': {'target': [], 'bioactivities': [], 'assay': []}, 'Bosutinib': {'target': [], 'bioactivities': [], 'assay': []}, 'Sgc-cbp30': {'target': [], 'bioactivities': [], 'assay': []}, 'Dovitinib': {'target': [], 'bioactivities': [], 'assay': []}, 'RN-486': {'target': [], 'bioactivities': [], 'a

In [113]:
target_not_found =[]
bioactivities_not_found =[]
assay_not_found=[]

for compound in compounds.keys():
    print(compound)
    target = targets_api.search(compound)
    bioactivities = protein_api.search(compound)
    assay = assay_api.search(compound)
    if len(target) != 0:
        compounds[compound]["target"].append(target)
    else:
        target_not_found.append(compound)
        compounds[compound]["target"].append("None")

    if len(bioactivities) != 0:
        compounds[compound]["bioactivities"].append(list(bioactivities[0]))
    else:
        bioactivities_not_found.append(compound)
        compounds[compound]["bioactivities"].append("None")
    
    if len(assay[0]) != 0:
        compounds[compound]["assay"].append(assay)
    else:
        assay_not_found.append(compound)
        compounds[compound]["assay"].append("None")
    

    print(f"Length and type of bioactivites object: {len(bioactivities)}, {type(bioactivities)}")
    print(bioactivities)
    print(f"Length and type of target object: {len(target)}, {type(target)}")
    print(target)
    print(target_not_found)
    print(bioactivities)

Tacalcitol
Length and type of bioactivites object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of target object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
['Tacalcitol']
[]
Penfluridol
Length and type of bioactivites object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of target object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
['Tacalcitol', 'Penfluridol']
[]
AVL-292
Length and type of bioactivites object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of target object: 1, <class 'chembl_webresource_client.query_set.QuerySet'>
[{'cross_references': [], 'organism': 'Homo sapiens', 'pref_name': 'G-292', 'score': 18.0, 'species_group_flag': False, 'target_chembl_id': 'CHEMBL614298', 'target_components': [], 'target_type': 'CELL-LINE', 'tax_id': 9606}]
['Tacalcitol', 'Penfluridol']
[]
HMN-214
Length and type of bioactivites object: 0, <class 'chembl_webresou

In [114]:
len(target_not_found)
len(bioactivities_not_found)
len(assay_not_found)
compounds

TypeError: 'NoneType' object cannot be interpreted as an integer

In [118]:
# Get target information from ChEMBL but restrict it to specified values only

chembl_name = []
chembl_none=[]
chembl_both = {}


for i in range(0, len(cmp)):
    name = compounds_api.search(cmp[i]).only('molecule_chembl_id')
    if name[0] == None:
        chembl_none.append(cmp[i])
        continue
    else:   
        chembl_name.append(name[0].get('molecule_chembl_id'))
        chembl_both[cmp[i]] = name[0].get('molecule_chembl_id')

print(chembl_name)
print(chembl_none)
print(chembl_both)


TypeError: quote_from_bytes() expected bytes

In [ ]:
targets = pd.DataFrame.from_records(cmp)
targets

,molecule_chembl_id
0,CHEMBL2105611
1,CHEMBL2105611


In [12]:
for compound in compounds:
    bioactivities = protein_api.search(compound)

    print(f"Length and type of bioactivities object: {len(bioactivities)}, {type(bioactivities)}")
    print(bioactivities)

Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]
Length and type of bioactivities object: 0, <class 'chembl_webresource_client.query_set.QuerySet'>
[]


KeyboardInterrupt: 

In [11]:
print(f"Length and type of first element: {len(bioactivities[0])}, {type(bioactivities[0])}")
bioactivities[0]

TypeError: object of type 'NoneType' has no len()

In [11]:
bioactivities_df = pd.DataFrame.from_dict(bioactivities)
print(f"DataFrame shape: {bioactivities_df.shape}")
bioactivities_df.head()

NameError: name 'bioactivities' is not defined

In [ ]:
bioactivities_df.drop(["units", "value"], axis=1, inplace=True)
bioactivities_df.head()

In [ ]:
bioactivities_df = bioactivities_df.astype({"standard_value": "float64"})
bioactivities_df.dropna(axis=0, how="any", inplace=True)
print(f"DataFrame shape: {bioactivities_df.shape}")
bioactivities_df.drop_duplicates("molecule_chembl_id", keep="first", inplace=True)
print(f"DataFrame shape: {bioactivities_df.shape}")
bioactivities_df.reset_index(drop=True, inplace=True)
bioactivities_df.head()